# Method-Specific Algorithms  

All of the algorithms below are Classical Algorithms. As such they can be augmented via the pulse-spectrum as an additional constraint as well as momentum to accelerated convergence. More details can be found in [Overview Algorithms](example_overview_algorithms.ipynb).

## FROG

### Vanilla  

The first published FROG algorithms. It is similar to the Generalized Projection Algorithm. However instaed of an iterative optimization of the Z-error, a simple integration along $\tau$ is used to generate an updated guess.  

$E'(t) = \int S'(\tau, t)\, d\tau$  

Below is a minimal working example applied to SHG-FROG. There are no tunable parameters.

In [ ]:
from pulsedjax.frog import Vanilla

vanilla = Vanilla(delay, frequency, trace, "shg")

population = vanilla.create_initial_population(5, "random")

final_result = vanilla.run(population, 50)

### LSGPA  

The Least-Squares Generalized-Projection Algorithm is quite similar to the Vanilla- and Generalized Projection Algorithms. However it works by applying a linear least-squares approach to the Z-error in order to generate a new update. This works since the signal field $S(\tau, t) = E(t)\cdot G(\tau, t)$ depends on the pulse only in a linear fashion. (If implicit dependencies are ignored.) Thus an update formula for the pulse can be derived:  

$E'(t) = \frac{\int S'(\tau, t)\cdot G^*(\tau, t)\, d\tau}{\int |G(\tau, t)|^2\, d\tau}$  

Below is a minimal working example applied to SHG-FROG. There are no tunable parameters.

In [ ]:
from pulsedjax.frog import LSGPA

lsgpa = LSGPA(delay, frequency, trace, "shg")

population = lsgpa.create_initial_population(5, "random")

final_result = lsgpa.run(population, 50)

### C-PCGPA

## Chirp-Scan

### Basic

## 2D-SI

### Direct-Reconstruction